#### Required Libraries

In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

#### Detecting the lanes

In [2]:
def region_of_interest(image):
    height=image.shape[0]
    triangle=np.array([[(250,height),(1000,height),(550,250)]])
    region=np.zeros_like(image)
    cv2.fillPoly(region,triangle,255)
    dup=cv2.bitwise_and(image,region)
    return dup

#### Detecting Straight Lines

In [3]:
def display_lines(image,lines):
    line_image=np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            cv2.line(line_image,(x1,y1),(x2,y2),(0,255,0),10)
    return line_image

#### Optimizing the lines

In [4]:
val1=[]
val2=[]
def default(left, right):
    globals()['val1']=left
    globals()['val2']=right
        

def averaged_slope_intercept(image,lines):
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        parameters=np.polyfit((x1,x2),(y1,y2),1)
        slope=parameters[0]
        intercept=parameters[1]
        if slope<0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
    left_avg=np.average(left_fit,axis=0)
    right_avg=np.average(right_fit,axis=0)
    if len(globals()['val1'])==0:
        default(left_avg,right_avg)
    left_line=new_coordinates(image,left_avg)
    right_line=new_coordinates(image,right_avg)
    return np.array([left_line,right_line])


def new_coordinates(image,parameters):
    try:
        m,b = parameters
    except TypeError:
        m,b = globals()['val1']
    y1=image.shape[0]
    y2=int(y1*2/3)
    x1=int((y1-b)/m)
    x2=int((y2-b)/m)
    return np.array([x1,y1,x2,y2])

#### Performing these operations on an image

In [5]:
image=cv2.imread('C:/Users/chhay/Downloads/test_image.jpg')

# Converting the image to gray level image
image2=np.copy(image)
gray_image=cv2.cvtColor(image2,cv2.COLOR_RGB2GRAY)

# Detecting the edges og the image
blur=cv2.GaussianBlur(gray_image,(5,5),0)
canny=cv2.Canny(blur,50,100)

# Determining the region of interest
cropped_image=region_of_interest(canny)

# Detecting the lines and optimizing them
lines=cv2.HoughLinesP(cropped_image,3,np.pi/180,100,np.array([]),minLineLength=45,maxLineGap=6)
average_lines=averaged_slope_intercept(image2,lines)
line_image=display_lines(image2,average_lines)

# To display line image and original image together
lines_on_image=cv2.addWeighted(line_image,0.6,image2,1,1)
cv2.imshow("Res",lines_on_image)
cv2.waitKey(0)

-1

#### Performing these operations on a video

In [6]:
cap=cv2.VideoCapture("C:/Users/chhay/Downloads/test2.mp4")
while(cap.isOpened()):
    ret,frame=cap.read() 
    if not ret:
        break
    gray_image=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    blur=cv2.GaussianBlur(gray_image,(5,5),0)
    canny_image=cv2.Canny(blur,50,100)
    
    # Determining the region of interest
    cropped_image=region_of_interest(canny_image)

    # Detecting the lines and optimizing them
    lines=cv2.HoughLinesP(cropped_image,3,np.pi/180,100,np.array([]),minLineLength=45,maxLineGap=6)
    average_lines=averaged_slope_intercept(frame,lines)
    line_image=display_lines(frame,average_lines)

    # To display line image and original image together
    lines_on_image=cv2.addWeighted(line_image,0.6,frame,1,1)
    cv2.imshow("Res",lines_on_image)
    k = cv2.waitKey(1) & 0xFF
    if k  == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

C:\Users\chhay\anaconda3\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\chhay\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
